In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import holoviews as hv
hv.extension('bokeh')
import bokeh
from bokeh.io import output_notebook, show
import bokeh.plotting as bp # figure, show, output_file
from talk_plottingutils import *
output_notebook()

import functools
import toolz

# Cell fate prediction in hematopoietic stem cells
[Prospective identification of hematopoietic lineage choice by deep learning](http://www.nature.com/nmeth/journal/v14/n4/full/nmeth.4182.html)    
Felix Buggenthin, Florian Buettner, Philipp S Hoppe, Max Endele, Manuel Kroiss, Michael Strasser, Michael Schwarzfischer, Dirk Loeffler, Konstantinos D Kokkaliaris, Oliver Hilsenbeck, Timm Schroeder, Fabian J Theis, Carsten Marr<p>
Nature Methods 14, 403–406 (2017)


[Code](https://github.com/QSCD/HematoFatePrediction) 

## Quick recap:
- murine blood stem cells differentiate into distinct cell types


<!--![Orkin1](orkin1.png)-->
![Orkin2](images/orkin2.png)
<img src="images/time_lapse_setup.png" alt="time_lapse" style="width: 200px;"/>

- continuous in vitro imaging of this process over several days

# Raw data
brightfield images

<img src="images/movie_positions.png" alt="Position layout" style="width: 400px;"/>
<img src="images/single_position.png" alt="single position" style="width: 400px;"/>

# Handling the data
a convenient wrapper around the time-lapse data:

    https://github.com/redst4r/movieTools.git
    
Should already be installed if you followed these [instructions](0_overview.ipynb)

In [ ]:
import movieTools.config
import movieTools.imageNormalizer as imageNormalizer
from movieTools.movie import Movie
movieTools.config.TTTDIR = '/home/michi/pythonProjects/deepLearning/Hemato_korea/data_small/'  # change that to your location!

%ls /home/michi/pythonProjects/deepLearning/Hemato_korea/data_small/

In [ ]:
movie = Movie('experiment3', verbose=False)  # 140206

print('Positions')
print(movie.get_all_positions())

print('\nTotal number of images')
len(movie.get_all_images())

lets look at a single raw image from position 54, timepoint 3330

**Note**: The downloaded data contains only timepoints 3300-3399 (usually, 1-5000)

In [ ]:
I = movie.loadimage(position=54, 
                    timepoint=3330, 
                    WL='w00',        # Brightfield
                    extension='png', # for backwards compatibility
                    normalizer=imageNormalizer.NoNormalizer())  # just load raw images, no background correction
plot_image_bokeh(I);

In [ ]:
# a couple of consecutive frames
tt = range(3330,3350)
img_list = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', normalizer=imageNormalizer.NoNormalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
hv_original_stack = hv_plot_stack(img_list, tt)

In [ ]:
%%output size=150
%%opts Image style(cmap='Greys_r')
hv_original_stack 

## Observations
- uneven illumination
- borders of the cover-slip
- dirt

### mean intensity over time

In [ ]:
def image_iterator(movie, position):
    "returns a generator over all images of that position"
    timepoints = [time for (pos, time, wl), fname in movie.get_all_images().items() if pos==position]
    
    timepoints = np.sort(timepoints)
    for t in timepoints:
        yield movie.loadimage(position, timepoint=t, WL='w00', extension='png', normalizer=imageNormalizer.NoNormalizer())      

In [ ]:
# might take 20 sec or so
average_intensity_over_time = [np.mean(I) for I in image_iterator(movie, position=54)]

In [ ]:
plt.plot(average_intensity_over_time)
plt.xlabel('Time')
plt.ylabel('avg. intensity')
plt.show()

### mean image

In [ ]:
Q = toolz.take(100, image_iterator(movie, position=54))  # just average across 100 images
# Q = image_iterator(movie, position=54)  # careful this loads alot of data into RAM
meanI= np.stack(Q).mean(0)

In [ ]:
plot_image_bokeh(meanI);  # plot_image_mpl(meanI); plt.grid(); plt.show()

# Background normalization
**Methods**

1. subtracing the time averaged image (get rid of static dirt)
2. subtract space averaged mean for each image (get rid of changing intensities)
3. more sophisticated: [BaSiC](https://github.com/QSCD/BaSiC). Takes care of all at once


## subtract the time average

**Load timepoint 2850 and normalize it by subtracting the temporal average**

In [ ]:
# our test image
I0 = movie.loadimage(position=54, timepoint=3310, WL='w00', extension='png', normalizer= imageNormalizer.NoNormalizer())

plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(I0, cmap=plt.cm.Greys_r)
plt.subplot(1,2,2)
plt.imshow(I0-meanI, cmap=plt.cm.Greys_r)
plt.show()

## BaSiC

    A BaSiC Tool for Background and Shading Correction of Optical Microscopy Images
    Tingying Peng, Kurt Thorn, Timm Schroeder, Lichao Wang, Fabian J Theis, Carsten Marr, Nassir Navab
    Nature Communication 8:14836 (2017)
    
    
    
$$I(x,t) = [B(t) \cdot I^{true}(x,t)] \cdot S(x) + D(x)$$
- $S(x)$ flatfield, uneven illumination
- $D(x)$ darkfield, camera offset, noise
- $B(t)$ time dependent baseline intensity (i.e. due to photobleaching)
- smoothness constrains on $S(x),D(x)$ (sparse in Fourier space)

In [ ]:
I0 = movie.loadimage(position=54, timepoint=3310, WL='w00', extension='png', 
                     normalizer= imageNormalizer.NoNormalizer())

I_basic = movie.loadimage(position=54, timepoint=3310, WL='w00', extension='png', 
                          normalizer= imageNormalizer.BaSiC_Normalizer())

plt.figure(figsize=(20,20))
plt.subplot(1,2,1)
plt.imshow(I0, cmap=plt.cm.Greys_r)
plt.subplot(1,2,2)
plt.imshow(I_basic, cmap=plt.cm.Greys_r)
plt.show()

# Compare to the original frames at the top of the notebook

** Normalize the entire stack of images from t=3430-3450**

In [ ]:
# a couple of consecutive frames
tt = range(3330,3350)
img_list_corrected = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', 
                                      normalizer=imageNormalizer.BaSiC_Normalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
hv_corrected_stack = hv_plot_stack(img_list_corrected, tt)

In [ ]:
%%output size=100
%%opts Image style(cmap='Greys_r')
hv_original_stack + hv_corrected_stack

# A larger timeframe

In [ ]:
tt = range(3300,3400, 2)
img_list = [movie.loadimage(position=54, timepoint=i, WL='w00', extension='png', 
                        normalizer=imageNormalizer.BaSiC_Normalizer())[:600:2,:600:2]
        for i in tt]

In [ ]:
%%output size=150
%%opts Image style(cmap='Greys_r')
hv_plot_stack(img_list, tt)